<a href="https://www.kaggle.com/code/eavprog/tr-portf-start-softmax?scriptVersionId=101376833" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Пробуем делать портфель

# Библиотеки

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

# Загрузка данных

In [ ]:
data = pd.read_csv('../input/accii-53/close_data.csv',index_col='Date')
data

# Изменение 

In [ ]:
d_data = data.pct_change()
d_data

# Симуляция

In [ ]:
?model.fit

In [ ]:
res = d_data.iloc[:,0].copy()
res[:] = 0

def custom_loss(y_true, y_pred):
    global y_t, y_p
#     y_t, y_p = y_true.eval() , y_pred.eval()
    
    return tf.math.exp( -tf.math.reduce_mean(y_true * y_pred)*y_true.shape[1] )

model = None

def get_distr(d_inp):
    distr = np.random.randn(d_inp.shape[1])
    distr = np.exp(distr)/sum(np.exp(distr))
    
    global model, inps, outs, inps_learn, outs_learn, pred
    
    if d_inp.shape[0] % 5 == 0:
        # print("model = None")
        model = None
    
    if d_inp.shape[0] < 300:
        distr = np.ones_like(distr) / len(distr)
    else:
        # разделение выборок
        inps = pd.concat([d_inp.shift(n) for n in range(20)],axis=1) # входы
        # outs = pd.concat([d_inp.shift(-n) for n in range(5)],axis=1) # выходы
        outs = d_inp.shift(-1)
        
        # ограничение размера обучающей выборки
        inps = inps.iloc[-300:,:]
        outs = outs.iloc[-300:,:]
        
        # выделяем непустые строки для обучения
        ind = (inps.isna().sum(axis=1) == 0) & (outs.isna().sum(axis=1) == 0)
        inps_learn = inps[ind]
        outs_learn = outs[ind]
        
        if model is None:
            # создание модели
            model = tf.keras.models.Sequential([
                layers.Dense(128, activation='relu',input_shape=(inps.shape[1],)),
                layers.Dropout(0.2),
                layers.Dense(d_inp.shape[1],activation='softmax')
            ])

            model.compile(optimizer="adam",loss=custom_loss)

            # обучение модели
            model.fit(inps_learn,outs_learn, 
                      epochs=1000, 
                      verbose = False, 
                      validation_split = 0.2, 
                      use_multiprocessing=True, 
                      shuffle=False,
                      callbacks=tf.keras.callbacks.EarlyStopping(patience=100,restore_best_weights=True,verbose=False))
            
            # print("model fited")
            
        # генерация прогноза
        
        distr = model.predict(inps.iloc[[-1],:])[0]
        
        # raise 0
        
        # pass

    return distr    

for n in range(len(res)):
    before = d_data.iloc[:n,:].copy()
    distr = get_distr(before)
    res.iloc[n] = (d_data.iloc[n,:] * distr).sum()
    if n % 50 == 0:
        print(res.index[n])


d_data_avg = d_data.fillna(0).mean(axis=1).mean()*100
d_portf_avg = res.mean()*100
(res*100).plot(title=f"изменение портфеля (дневная доходность портфеля {d_portf_avg:.3}%, равномерного {d_data_avg:.3}%)",
         figsize=(15,10),grid=True,ylabel='%')

plt.figure(figsize=(15,10))
((res+1).cumprod()*100).plot(label='портфель')
((d_data.fillna(0).mean(axis=1)+1).cumprod()*100).plot(label='равномерный')
plt.title('накопление портфеля')
plt.grid()
plt.ylabel('%')
plt.legend()
plt.show()